In [62]:
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mne
import os

import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader, TensorDataset, Subset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
import pandas as pd
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
from utils import load_dataset_EEG
from tqdm import tqdm
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from utils import save_plot_with_timestamp

In [18]:
headers = [
    "AFF1h", "F7", "FC5", "C3", "T7", "TP9", "Pz", "P3", "P7", "O1", "O2", "P8", "P4", "TP10", "Cz", "C4", "T8", "FC6", "FCz", "F8", "AFF2h", "GSR", "EKG", "arousal_score", "valence_score"]

file = '/Users/calebjonesshibu/Desktop/tom/derived/draft_2023_06_05_11/eeg/exp_2022_10_04_09/leopard_affective_individual_physio_task.csv'
df = pd.read_csv(file)
imac = os.path.basename(file).split('_')[0]
df.drop(columns=['unix_time', 'task_time', 'task_monotonic_time', 'task_human_readable_time', 'task_subject_id', 'seconds_since_start', 'human_readable_time', imac, 'task_index', 'experiment_id'], inplace=True)
if 'task_Unnamed: 0' in df.columns:
    df.drop(columns='task_Unnamed: 0', inplace=True)

df.loc[df['task_event_type'].isin(['intermediate_selection','show_cross_screen', 'show_image', 'show_rating_screen']), ['task_arousal_score', 'task_valence_score']] = np.nan
df[['task_image_path', 'task_arousal_score', 'task_valence_score', 'task_event_type']] = df[['task_image_path', 'task_arousal_score', 'task_valence_score', 'task_event_type']].fillna(method='bfill')
df.drop(columns=['task_image_path', 'task_event_type'], inplace=True)
df.dropna(inplace=True)
df.columns = [None] * len(df.columns)

df = df.set_axis(headers, axis=1)
df_EEG = df.iloc[:, :23]

In [42]:
ignore_channels = ['AUX_GSR', 'AUX_EKG']
channel_names = df_EEG.columns[:21].to_list()
channel_types = ['eeg' if ch not in ignore_channels else 'misc' for ch in channel_names]

info = mne.create_info(ch_names=channel_names, sfreq=500, ch_types=channel_types)
montage = mne.channels.make_standard_montage('standard_1005')
info.set_montage(montage)
raw = mne.io.RawArray(np.array(df_EEG.iloc[:, :21]).T, info)

Creating RawArray with float64 data, n_channels=21, n_times=89100
    Range : 0 ... 89099 =      0.000 ...   178.198 secs
Ready.


In [43]:
frequency_bands = {
            'delta': (1, 4),
            'theta': (4, 8),
            'alpha': (8, 14),
            'beta': (14, 31),
            'gamma': (31, 50)
        }
filtered_band_data_dict = {}
for band_name, (l_freq, h_freq) in frequency_bands.items():
    filtered_band_data = raw.copy().filter(l_freq=l_freq, h_freq=h_freq, skip_by_annotation='edge', picks=['eeg'])
    filtered_band_data_dict[band_name] = filtered_band_data 

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 4 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 4.00 Hz
- Upper transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 5.00 Hz)
- Filter length: 1651 samples (3.302 s)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 4 - 8 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 4.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 3.00 Hz)
- Upper passband edge: 8.00 

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  21 out of  21 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  21 out of  21 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 31 - 50 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 31.00
- Lower transition bandwidth: 7.75 Hz (-6 dB cutoff frequency: 27.12 Hz)
- Upper passband edge: 50.00 Hz
- Upper transition bandwidth: 12.50 Hz (-6 dB cutoff frequency: 56.25 Hz)
- Filter length: 213 samples (0.426 s)



[Parallel(n_jobs=1)]: Done  21 out of  21 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  21 out of  21 | elapsed:    0.0s finished


In [67]:
filtered_band_data_dict

{'delta': <RawArray | 21 x 89100 (178.2 s), ~14.3 MB, data loaded>,
 'theta': <RawArray | 21 x 89100 (178.2 s), ~14.3 MB, data loaded>,
 'alpha': <RawArray | 21 x 89100 (178.2 s), ~14.3 MB, data loaded>,
 'beta': <RawArray | 21 x 89100 (178.2 s), ~14.3 MB, data loaded>,
 'gamma': <RawArray | 21 x 89100 (178.2 s), ~14.3 MB, data loaded>}

In [74]:
data  = pd.DataFrame(filtered_band_data_dict['beta'].get_data().T)
data['arousal_score'] = df['arousal_score'].values
data['valence_score'] = df['valence_score'].values

In [138]:
def classify_LSTM_Affective_Individual_Task_EEG(path, hidden_size, num_epochs, batch_size, learning_rate):

    # Create the output folder if it doesn't exist
    output_folder = 'output'
    if not os.path.exists(output_folder):
        try:
            os.makedirs(output_folder)
        except OSError as e:
            print(f"Error creating output folder: {e}")
            return
        
    # Load dataset
    merged_df = data #load_dataset_EEG(path)

    # Check if CUDA is available
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Preprocess data
    features = merged_df.iloc[:, :-2].values
    arousal_score = LabelEncoder().fit_transform(merged_df.iloc[:, -2] + 2)  # Mapping -2 -> 0, -1 -> 1, 0 -> 2, 1 -> 3, 2 -> 4
    valence_score = LabelEncoder().fit_transform(merged_df.iloc[:, -1] + 2)  # Same mapping for valence_score
    targets = list(zip(arousal_score, valence_score))

    # Hyperparameters
    input_size = features.shape[1]
    num_classes = 5  # Classes representing -2, -1, 0, 1, 2
    num_folds = 5

    # Create DataLoaders
    dataset = TensorDataset(torch.tensor(features).float().to(device), torch.tensor(targets).long().to(device))
    kfold = KFold(n_splits=num_folds, shuffle=True, random_state=42)

    # Define model
    class LSTM(nn.Module):
        def __init__(self, input_size, hidden_size, num_classes):
            super(LSTM, self).__init__()
            self.hidden_size = hidden_size
            self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
            self.fc = nn.Linear(hidden_size, num_classes)
        
        def forward(self, x):
            out, _ = self.lstm(x)
            out = self.fc(out[:, -1, :])
            return out

    # Initialize model, loss, and optimizer
    model = LSTM(input_size, hidden_size, num_classes).to(device)  # Move the model to the GPU
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Perform k-fold cross-validation
    fold_losses = []
    fold_accuracies = []

    for fold, (train_indices, test_indices) in enumerate(kfold.split(dataset)):
        print(f"Fold {fold+1}/{num_folds}")

        # Split data into train and test sets for the current fold
        train_data = Subset(dataset, train_indices)
        test_data = Subset(dataset, test_indices)
        train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
        test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

        # Training
        model.train()
        for epoch in range(num_epochs):
            progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}")
            for i, (inputs, targets) in enumerate(progress_bar):
                inputs = inputs.view(-1, 1, input_size)
                targets_arousal = targets[:, 0]
                targets_valence = targets[:, 1]

                outputs = model(inputs)

                loss_arousal = criterion(outputs, targets_arousal)
                loss_valence = criterion(outputs, targets_valence)

                loss = loss_arousal + loss_valence  # Total loss

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                progress_bar.set_postfix(loss=loss.item())

        fold_losses.append(loss.item())
        print(f"Loss for Fold {fold+1}: {fold_losses[-1]}")

        # Testing
        model.eval()
        correct_arousal, correct_valence = 0, 0
        total = 0

        with torch.no_grad():
            for inputs, targets in test_loader:
                inputs = inputs.view(-1, 1, input_size)
                targets_arousal = targets[:, 0]
                targets_valence = targets[:, 1]

                outputs = model(inputs)

                _, predicted_arousal = torch.max(outputs.data, 1)
                _, predicted_valence = torch.max(outputs.data, 1)

                total += targets.size(0)
                correct_arousal += (predicted_arousal == targets_arousal).sum().item()
                correct_valence += (predicted_valence == targets_valence).sum().item()

        accuracy_arousal = 100 * correct_arousal / total
        accuracy_valence = 100 * correct_valence / total
        fold_accuracies.append((accuracy_arousal, accuracy_valence))

    # Print average accuracy and standard deviation across folds
    arousal_accuracies, valence_accuracies = zip(*fold_accuracies)
    print("Average accuracy for arousal_score:", np.mean(arousal_accuracies))
    print("Standard deviation for arousal_score:", np.std(arousal_accuracies))
    print("Average accuracy for valence_score:", np.mean(valence_accuracies))
    print("Standard deviation for valence_score:", np.std(valence_accuracies))

    # Print the average loss per fold.
    print(f"Average loss per fold: {np.mean(fold_losses)}")
    print(f"Standard deviation of loss per fold: {np.std(fold_losses)}")

    arousal_cm = confusion_matrix(targets_arousal.cpu(), predicted_arousal.cpu())
    valence_cm = confusion_matrix(targets_valence.cpu(), predicted_valence.cpu())
    # Define the class names (assuming -2 to 2 for arousal and valence scores)
    class_names = [-2, -1, 0, 1, 2]

    # Plotting confusion matrix for arousal
    plt.figure(figsize=(20, 14))
    sns.heatmap(arousal_cm, annot=True, fmt='d', xticklabels=class_names, yticklabels=class_names, cmap='Blues')
    plt.title(f'EEG: Confusion Matrix for Arousal\nHidden Size: {hidden_size}, Batch Size: {batch_size}, Learning Rate: {learning_rate}, Epochs: {num_epochs}, Accuracy: {accuracy_arousal:.2f}%, std: {np.std(arousal_accuracies):.2f}%, loss: {np.mean(fold_losses):.2f}, std: {np.std(fold_losses):.2f}')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    save_plot_with_timestamp(plt, 'confusion_matrix_arousal', output_folder)

    # Plotting confusion matrix for valence
    plt.figure(figsize=(20, 14))
    sns.heatmap(valence_cm, annot=True, fmt='d', xticklabels=class_names, yticklabels=class_names, cmap='Blues')
    plt.title(f'EEG: Confusion Matrix for Valence\nHidden Size: {hidden_size}, Batch Size: {batch_size}, Learning Rate: {learning_rate}, Epochs: {num_epochs}, Accuracy: {accuracy_valence:.2f}%, std: {np.std(valence_accuracies):.2f}%, loss: {np.mean(fold_losses):.2f}, std: {np.std(fold_losses):.2f}')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    save_plot_with_timestamp(plt, 'confusion_matrix_valence', output_folder)

# if __name__ == "__main__":
#     parser = argparse.ArgumentParser(
#         description="Post experiment script for xdf to csv file conversion"
#     )
#     parser.add_argument(
#         "--p", required=True, help="Path to the directory with the derived affective task data"
#     )
#     parser.add_argument(
#         "--hidden_size", type=int, default=256, help="Hidden size for the LSTM model"
#     )
#     parser.add_argument(
#         "--num_epochs", type=int, default=100, help="Number of epochs for training"
#     )
#     parser.add_argument(
#         "--batch_size", type=int, default=256, help="Batch size for training"
#     )
#     parser.add_argument(
#         "--learning_rate", type=float, default=0.001, help="Learning rate for optimizer"
#     )

#     args = parser.parse_args()
#     path = args.p
#     hidden_size = args.hidden_size
#     num_epochs = args.num_epochs
#     batch_size = args.batch_size
#     learning_rate = args.learning_rate

#     sys.exit(classify_LSTM_Affective_Individual_Task_EEG(path, hidden_size, num_epochs, batch_size, learning_rate))


In [139]:
classify_LSTM_Affective_Individual_Task_EEG(_, 120, 2, 256, 0.01)

Input size: 21
Fold 1/5


Epoch 2/2: 100%|██████████| 279/279 [00:02<00:00, 120.87it/s, loss=2.56]


Loss for Fold 1: 2.5571048259735107
Fold 2/5


Epoch 2/2: 100%|██████████| 279/279 [00:02<00:00, 128.63it/s, loss=2.51]


Loss for Fold 2: 2.512396812438965
Fold 3/5


...

Loss for Fold 3: 2.561903953552246
Fold 4/5


Epoch 2/2: 100%|██████████| 279/279 [00:02<00:00, 131.29it/s, loss=2.66]


Loss for Fold 4: 2.6559977531433105
Fold 5/5


Epoch 2/2: 100%|██████████| 279/279 [00:02<00:00, 120.83it/s, loss=2.65]


Loss for Fold 5: 2.647057294845581
Average accuracy for arousal_score: 35.87542087542088
Standard deviation for arousal_score: 0.3162116331817881
Average accuracy for valence_score: 34.931537598204265
Standard deviation for valence_score: 0.3651883854351852
Average loss per fold: 2.5868921279907227
Standard deviation of loss per fold: 0.05559978236070936


test

In [114]:
file = '/Users/calebjonesshibu/Desktop/tom/derived/draft_2023_06_05_11/eeg/exp_2022_10_04_09/leopard_affective_individual_physio_task.csv'
df = pd.read_csv(file)
imac = os.path.basename(file).split('_')[0]
df.drop(columns=['task_image_path', 'task_valence_score', 'task_arousal_score', 'task_event_type', 'task_time', 'task_monotonic_time', 'task_human_readable_time', 'task_subject_id', 'seconds_since_start', 'human_readable_time', imac, 'task_index', 'experiment_id'], inplace=True)
if 'task_Unnamed: 0' in df.columns:
    df.drop(columns='task_Unnamed: 0', inplace=True)

In [125]:
# Convert the 'unix_time' column to datetime
# df['unix_time'] = pd.to_datetime(df['unix_time'], unit='s')
df.dropna(inplace=True)
# Convert the DataFrame to MNE Raw object
raw = mne.io.RawArray(df.T.values[1:], info=mne.create_info(ch_names=df.columns[1:].to_list(), sfreq=1/(df['unix_time'].diff().mean().total_seconds()),
                                                              ch_types=['eeg']*len(df.columns[1:])), verbose=False)

sfreq = 500

# Specify the desired sampling rate after downsampling
new_sampling_rate = 100  # New sampling rate in Hz

# Downsample the data
raw_resampled = raw.resample(new_sampling_rate)

# Before downsampling
unix_times = df['unix_time'].values

# After downsampling
downsample_ratio = int(sfreq / new_sampling_rate)
downsampled_unix_times = unix_times[::downsample_ratio]

In [126]:
len(downsampled_unix_times), raw_resampled.get_data().shape

(17820, (23, 17820))

In [127]:
df.shape

(89100, 24)

In [94]:
# Calculate the mean time difference between consecutive unix_time values
mean_time_diff = df['unix_time'].diff().mean().total_seconds()

# Specify the desired new sampling rate after downsampling
new_sampling_rate = 100  # New sampling rate in Hz

# Calculate the downsampling factor
downsampling_factor = int(1 / (new_sampling_rate * mean_time_diff))

# Downsample the EEG data
downsampled_data = df.iloc[::downsampling_factor]

# Downsample the unix_time column
downsampled_unix_time = df['unix_time'].iloc[::downsampling_factor]

In [95]:
downsampled_data

,unix_time,leopard_AFF1h,leopard_F7,leopard_FC5,leopard_C3,leopard_T7,leopard_TP9,leopard_Pz,leopard_P3,leopard_P7,...,leopard_TP10,leopard_Cz,leopard_C4,leopard_T8,leopard_FC6,leopard_FCz,leopard_F8,leopard_AFF2h,leopard_AUX_GSR,leopard_AUX_EKG
1,2022-10-04 18:16:40.677009152,0.015914,-0.011380,0.008687,0.008589,-0.014979,0.011379,0.007745,0.011237,0.013657,...,0.012609,0.017386,0.010032,0.006250,0.011365,0.022363,0.009064,0.011718,-0.356466,0.057533
6,2022-10-04 18:16:40.687009792,0.018926,-0.008364,0.011702,0.011599,-0.011972,0.014389,0.010744,0.014244,0.016665,...,0.015631,0.020385,0.013049,0.009267,0.014381,0.025368,0.012104,0.014726,-0.092487,0.058417
11,2022-10-04 18:16:40.697010432,0.017012,-0.010279,0.009790,0.009687,-0.013884,0.012489,0.008850,0.012342,0.014766,...,0.013702,0.018484,0.011149,0.007362,0.012477,0.023460,0.010158,0.012820,-0.158780,0.057018
16,2022-10-04 18:16:40.707010816,0.012932,-0.014368,0.005701,0.005600,-0.017977,0.008366,0.004782,0.008256,0.010655,...,0.009616,0.014411,0.007066,0.003271,0.008389,0.019380,0.006113,0.008744,-0.233194,0.058511
21,2022-10-04 18:16:40.717011200,0.012278,-0.015031,0.005041,0.004940,-0.018648,0.007700,0.004123,0.007590,0.009988,...,0.008943,0.013753,0.006396,0.002595,0.007719,0.018724,0.005414,0.008084,0.012998,0.056775
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89087,2022-10-04 18:19:38.857335040,0.017070,-0.010044,0.009765,0.009555,-0.013778,0.012521,0.008778,0.012267,0.014683,...,0.013703,0.018499,0.011129,0.007221,0.012467,0.023662,0.010141,0.012730,0.014982,0.058009
89092,2022-10-04 18:19:38.867335680,0.013693,-0.013424,0.006390,0.006178,-0.017163,0.009145,0.005413,0.008889,0.011298,...,0.010313,0.015132,0.007748,0.003837,0.009080,0.020285,0.006728,0.009356,-0.154783,0.056983
89097,2022-10-04 18:19:38.877336576,0.014111,-0.013010,0.006800,0.006593,-0.016750,0.009552,0.005824,0.009298,0.011718,...,0.010727,0.015544,0.008159,0.004252,0.009497,0.020701,0.007155,0.009770,-0.124649,0.058420
89102,2022-10-04 18:19:38.887336704,0.017884,-0.009230,0.010576,0.010362,-0.012988,0.013326,0.009578,0.013063,0.015485,...,0.014500,0.019305,0.011929,0.008019,0.013266,0.024472,0.010937,0.013538,-0.032252,0.056823


In [96]:
downsampled_unix_time

1       2022-10-04 18:16:40.677009152
6       2022-10-04 18:16:40.687009792
11      2022-10-04 18:16:40.697010432
16      2022-10-04 18:16:40.707010816
21      2022-10-04 18:16:40.717011200
                     ...             
89087   2022-10-04 18:19:38.857335040
89092   2022-10-04 18:19:38.867335680
89097   2022-10-04 18:19:38.877336576
89102   2022-10-04 18:19:38.887336704
89107   2022-10-04 18:19:38.897337088
Name: unix_time, Length: 17820, dtype: datetime64[ns]